## Preprocessing

In [38]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd 

In [ ]:
def GetFilePath(filename):
    
    # If we're running in Colab, read from
    # the default upload directory.
    if 'google.colab' in str(get_ipython()):
        directory = './'
    else:
        directory = '../Resources/'
        
    filepath = directory + filename
    
    return filepath

In [ ]:
# If we're in Colab, prompt the user to import files
if 'google.colab' in str(get_ipython()):
    from google.colab import files
    uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [ ]:
#  Import and read the charity_data.csv.
filepath = GetFilePath('charity_data.csv')
application_df = pd.read_csv(filepath)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [30]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL']]

In [31]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION               6
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [32]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T25', 'T14', 'T29', 'T15', 'T17', 'T2', 'T12', 'T13', 'T9']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [34]:
# Look at CLASSIFICATION value counts for binning
counts = charity_reduced_df['CLASSIFICATION'].value_counts()
counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [35]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ['C3200','C1246','C1234','C1256','C1267','C0','C1260','C2710', 'C2190','C4200','C2600','C5200','C1370','C1248','C6100','C1820','C1900', 'C1236','C3700', 'C2570','C1580','C1245','C2500','C1570','C1283','C2380','C1732','C1728','C2170','C4120','C8210','C2561','C4500', 'C2150', 'C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [36]:
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [40]:
# Convert categorical data to numeric **Per our instructor we can use OneHotEncoder
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, axis='columns')
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [42]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [43]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [58]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  108
hidden_nodes_layer2 = 108

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 108)               4752      
                                                                 
 dense_10 (Dense)            (None, 108)               11772     
                                                                 
 dense_11 (Dense)            (None, 1)                 109       
                                                                 
Total params: 16,633
Trainable params: 16,633
Non-trainable params: 0
_________________________________________________________________


In [59]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [60]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5726 - accuracy: 0.7213
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5574 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5537 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7323
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5496 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7330
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5472 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5464 - accura

In [61]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5594 - accuracy: 0.7261 - 466ms/epoch - 2ms/step
Loss: 0.5594192743301392, Accuracy: 0.726064145565033


In [64]:
# Export our model to HDF5 file
# Save the model
nn.save("AlphabetSoupCharity_Optimization.h5")

In [65]:
## Optimization Trial 1: Add more Nuerons

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  129
hidden_nodes_layer2 = 129

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 129)               5676      
                                                                 
 dense_13 (Dense)            (None, 129)               16770     
                                                                 
 dense_14 (Dense)            (None, 1)                 130       
                                                                 
Total params: 22,576
Trainable params: 22,576
Non-trainable params: 0
_________________________________________________________________


In [66]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
# Save the model
nn1.save("AlphabetSoupCharity_Optimization1.h5")

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5680 - accuracy: 0.7231
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5558 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7319
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7335
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5470 - accuracy: 0.7342
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accura

In [67]:
## Optimization Trial 2: Add more hidden layers

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  108
hidden_nodes_layer2 = 108
hidden_nodes_layer3 = 108

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 108)               4752      
                                                                 
 dense_16 (Dense)            (None, 108)               11772     
                                                                 
 dense_17 (Dense)            (None, 108)               11772     
                                                                 
 dense_18 (Dense)            (None, 1)                 109       
                                                                 
Total params: 28,405
Trainable params: 28,405
Non-trainable params: 0
_________________________________________________________________


In [68]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
# Save the model
nn2.save("AlphabetSoupCharity_Optimization2.h5")

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5699 - accuracy: 0.7229
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5554 - accuracy: 0.7317
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5527 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5512 - accuracy: 0.7328
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5499 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5471 - accuracy: 0.7350
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7353
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5461 - accura

In [69]:
## Optimization Trial 3: Add Epochs

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  108
hidden_nodes_layer2 = 108

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 108)               4752      
                                                                 
 dense_20 (Dense)            (None, 108)               11772     
                                                                 
 dense_21 (Dense)            (None, 1)                 109       
                                                                 
Total params: 16,633
Trainable params: 16,633
Non-trainable params: 0
_________________________________________________________________


In [70]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=200)

# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
# Save the model
nn3.save("AlphabetSoupCharity_Optimization3.h5")

Epoch 1/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5710 - accuracy: 0.7235
Epoch 2/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5571 - accuracy: 0.7308
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7313
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7329
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7322
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7322
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7332
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5470 - accuracy: 0.7345
Epoch 9/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7356
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accura

In [71]:
## Optimization Trial 4: Try different activation functions

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  108
hidden_nodes_layer2 = 108

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 108)               4752      
                                                                 
 dense_23 (Dense)            (None, 108)               11772     
                                                                 
 dense_24 (Dense)            (None, 1)                 109       
                                                                 
Total params: 16,633
Trainable params: 16,633
Non-trainable params: 0
_________________________________________________________________


In [72]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn4.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
# Save the model
nn4.save("AlphabetSoupCharity_Optimization4.h5")

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5773 - accuracy: 0.7191
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5611 - accuracy: 0.7274
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5561 - accuracy: 0.7308
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7304
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5521 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5506 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5497 - accuracy: 0.7325
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7326
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7329
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accura